In [4]:
#install tensorflow gpu
!pip install tensorflow_gpu==1.15.0

     |████████████████████████████████| 411.5 MB 7.6 kB/s 
     |████████████████████████████████| 50 kB 6.3 MB/s 
     |████████████████████████████████| 503 kB 59.7 MB/s 
     |████████████████████████████████| 3.8 MB 45.6 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=ec87aebbb4d8d8efc4c4de1b18b558ccdc265561716af42d9cd85b176c7eb139
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succe

In [5]:
!pip install 'h5py==2.10.0'

     |████████████████████████████████| 2.9 MB 27.0 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.7.0 requires tensorboard~=2.6, but you have tensorboard 1.15.0 which is incompatible.
tensorflow 2.7.0 requires tensorflow-estimator<2.8,~=2.7.0rc0, but you have tensorflow-estimator 1.15.1 which is incompatible.


In [6]:
#Import libraries
import numpy as np  
import pandas as pd 
import re 
import nltk 
import tensorflow as tf
import os
import pickle
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K         
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

In [21]:
# Download stopwords from nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
ROOT_PATH = "/content/drive/MyDrive/news_dataset_csv"
DATA_PATH = os.path.join(ROOT_PATH,"train-stats.csv")
MODEL_PATH = os.path.join(ROOT_PATH, "Text_summarization_model.h5")
X_TOKENIZER_PATH = os.path.join(ROOT_PATH, "x_tokenizer.pickle")
Y_TOKENIZER_PATH = os.path.join(ROOT_PATH, "y_tokenizer.pickle")
max_len_text=160
max_len_summary=30

In [13]:
with open(X_TOKENIZER_PATH, 'rb') as handle:
    x_tokenizer = pickle.load(handle)

with open(Y_TOKENIZER_PATH, 'rb') as handle:
    y_tokenizer = pickle.load(handle)

In [14]:
x_voc_size = len(x_tokenizer.word_index) + 1
y_voc_size = len(y_tokenizer.word_index) + 1

In [49]:
text_index_to_word = x_tokenizer.index_word 
summary_index_to_word = y_tokenizer.index_word 
summary_word_to_index = y_tokenizer.word_index

In [15]:
class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state
            inputs: (batchsize * 1 * de_in_dim)
            states: (batchsize * 1 * de_latent_dim)
            """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch size * en_seq_len * latent_dim
            W_a_dot_s = K.dot(encoder_out_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>', U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            if verbose:
                print('Ws+Uh>', Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

In [16]:
K.clear_session() 
latent_dim = 500 

# Encoder 
encoder_inputs = Input(shape=(max_len_text,)) 
enc_emb = Embedding(x_voc_size, latent_dim,trainable=True)(encoder_inputs) 

#LSTM 1 
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True) 
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb) 

#LSTM 2 
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True) 
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1) 

#LSTM 3 
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True) 
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2) 

# Set up the decoder. 
decoder_inputs = Input(shape=(None,)) 
dec_emb_layer = Embedding(y_voc_size, latent_dim,trainable=True) 
dec_emb = dec_emb_layer(decoder_inputs) 

#LSTM using encoder_states as initial state
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c]) 

#Attention Layer
attn_layer = AttentionLayer(name='attention_layer') 
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs]) 

# Concat attention output and decoder LSTM output 
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

# Concat attention output and decoder LSTM output 
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#Dense layer
decoder_dense = TimeDistributed(Dense(y_voc_size, activation='softmax')) 
decoder_outputs = decoder_dense(decoder_concat_input) 

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs) 

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [17]:
# encoder inference
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# decoder inference
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_len_text,latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs)

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_inf_concat)

# Final decoder model
decoder_model = Model(
[decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
[decoder_outputs2] + [state_h2, state_c2])

In [18]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))

    # Chose the 'start' word as the first word of the target sequence
    target_seq[0, 0] = summary_word_to_index['start']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, 1:]) + 1 #get index of highest probable predicted word
        #print(sampled_token_index)
        sampled_token = summary_index_to_word[sampled_token_index] #get word from inde

        if(sampled_token != 'end'): #check if word is not 'end'
          decoded_sentence += ' ' + sampled_token #concatenate predicted word to the decoded sentence
          if len(decoded_sentence.split()) >= (max_len_summary-1):
            stop_condition = True
            # Exit condition: If predicted word is 'end' or we have predicted maximum words in a sentence then stop
        else:
          stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [24]:
def preprocess_texts(text):
    text = re.sub(r'\([^)]*\)', '', text)    # replace special characters with empty string
    text = re.sub('"','', text)              # replace " with empty string
    text = re.sub(r"'s\b\n","",text)         # replace 's, \b, \n with empty string
    text = re.sub("[^a-zA-Z]", " ", text)    # replace words with other than alphabets with space
    text = text.lower()                      # transform text into lower case

    # split the text into list of words and select only those words which are not stopwords by removing trailing and leading spaces
    tokens = [w.strip() for w in text.split() if not w in stop_words] 

    #select only those words which is atleast 3 character long
    long_words = []
    for i in tokens:
        if len(i) >= 3:                  #removing short word
            long_words.append(i)
    return (" ".join(long_words)).strip()

In [54]:
texts = ["HAMBURG, Germany\n\t, June 3  As he left the soccer field after a club match in the eastern German city of Halle on March 25, the Nigerian forward Adebowale Ogungbure was spit upon, jeered with racial remarks and mocked with monkey noises. In rebuke, he placed two fingers under his nose to simulate a Hitler mustache and thrust his arm in a Nazi salute. Marc Zoro, right, an Ivory Coast native, was a target of racial slurs from the home fans in Messina, Italy. Adriano, a star with Inter Milan, tried to persuade him to stay on the field. From now until its conclusion on July 9, Jeff Z. Klein and other staff members of The Times and International Herald Tribune will track the world's most popular sporting event. Your guide to the games in Germany: teams, rosters, schedules, statistics, venues and more. In April, the American defender Oguchi Onyewu, playing for his professional club team in Belgium, dismissively gestured toward fans who were making simian chants at him. Then, as he went to throw the ball inbounds, Onyewu said a fan of the opposing team reached over\n\t a barrier and punched him in the face. International soccer has been plagued for years by violence among fans, including racial incidents. But FIFA, soccer's Zurich-based world governing body, said there has been a recent surge in discriminatory behavior toward blacks by fans and other players, an escalation that has dovetailed with the signing of more players from Africa and Latin America by elite European clubs. This "'deplorable trend,'" as FIFA has called it, now threatens to embarrass the sport on its grandest stage, the World Cup, which opens June 9 for a monthlong run in 12 cities around Germany. More than 30 billion cumulative television viewers are expected to watch part of the competition and Joseph S. Blatter, FIFA's president, has vowed to crack down on racist behavior during the tournament.", 'russia russia']

In [55]:
def predictions(texts):
  ''' Input : list of texts
      Output : list of summary
  '''
  stop_words = set(stopwords.words('english'))  #Use set of stopwords from english language
  predicted_summary = []
  for text in texts:
    text = preprocess_texts(text)
    text = x_tokenizer.texts_to_sequences([text])
    text = pad_sequences(text,  maxlen=max_len_text, padding='post') #It add zeroes to the end of sequence upto max length.
    summary = decode_sequence(text.reshape(1,max_len_text))
    predicted_summary.append(summary)
  return predicted_summary

In [56]:
predictions(texts)

[' requests adorns antiblackness clinch clinch medallion cleanliness backflip ghostbustersbuilding thaw detachment surgeons socialflow teenage borer domingo blockers semitism rdeyegirl rdeyegirl lavigne lavigne rugasira ruta sayig winterized philippon raised visalia',
 ' wayhaught housebuilders zalora sheldon fc librettist lisa ith propped irlan pai bukh bukh bukh bukh shutdowns plishka assets handicapping hewson issuing shorty castparts gryffindor mingardi thrombosis netblazr lacerations irreconcilable']